In [1]:
from yasearch.etc.schema import Document
import numpy as np
from loguru import logger
from tqdm import tqdm
from pathlib import Path
import os
import polars as pl
import uuid
from pathlib import Path
import simplejson as json
from typing import Optional, List, Dict
from weaviate.classes.query import MetadataQuery

In [2]:
!pip freeze | grep weaviate

weaviate-client==4.5.4


In [3]:
from transformers import AutoModel, AutoTokenizer
from torch import Tensor
import torch
from torch.functional import F
from functools import partial
from more_itertools import chunked
from yasearch.processing import loader, igniset

In [4]:
model_name_or_path = "intfloat/multilingual-e5-base"

In [5]:
model = AutoModel.from_pretrained(model_name_or_path)
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)

/home/justatom/ISpell/envs/polaroids/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [6]:
torch.cuda.is_available()

True

In [7]:
def prefix(x, pref:str):
    return pref.strip() + " " + x

In [8]:
def ignite_dataset(where) -> List[Dict]:
    docs = None
    with open(str(Path(where))) as fin:
        docs = json.load(fin)
    return docs

In [9]:
docs = ignite_dataset(where=Path(os.getcwd()) / ".data" / "polaroids.ai.data.json")

In [10]:
pl_data = pl.from_dicts(docs)

In [11]:
pl_data.head()

content,title,author,type,has_image,img_path,speaker,query
str,str,str,str,bool,str,str,str
"""В реалисте вер…","""Братья Карамаз…","""Ф.М. Достоевск…","""book""",false,null,null,null
"""Жизнь — это ми…","""Человек в футл…","""Антон Павлович…","""book""",false,null,null,null
"""Нет, не так. К…","""Метро 2033""","""Дмитрий Глухов…","""book""",true,"""./img/metro203…",null,null
"""Станьте солнце…","""Преступление и…","""Ф.М.Достоевски…","""book""",true,"""./img/crimeand…",null,null
"""Когда-то он бы…","""Джон Уик 3""","""...""","""movie""",true,"""./img/johnwick…",null,null


In [12]:
def tokenize(input_texts, tokenizer):
    batch_dict = tokenizer(input_texts, max_length=512, padding=True, truncation=True, return_tensors='pt')
    return batch_dict

In [13]:
def average_pool(last_hidden_states: Tensor,
                 attention_mask: Tensor) -> Tensor:
    last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
    return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]

In [14]:
documents = [d.get("content") for d in docs]
labels = [d.get("title") for d in docs]

In [36]:
tokenize(documents, tokenizer=tokenizer)["input_ids"].shape

torch.Size([616, 512])

In [16]:
def generate_unique_id(text, namespace_uuid='91461c99-f89d-49d2-af96-d8e2e14e9b58'):
    """
    
    Args:
    text (str): Textual content for generating fixed UUID.
    namespace_uuid (str): Namespace used by https://rethinkdb.com.
    
    Returns:
    str: Строковое представление детерминированного UUID.
    """
    namespace = uuid.UUID(namespace_uuid)
    
    deterministic_uuid = uuid.uuid5(namespace, text)
    
    return str(deterministic_uuid)

In [17]:
def ignite_vectors_by(model, docs, labels: Optional[List[str]] = None, pref:str="", norm: bool = True, batch_size:int = 2, device:str = "cpu"):
    wrapped_docs = []
    if labels is not None:
        it = chunked(zip(docs, labels), n=batch_size)
    else:
        it = chunked(docs, n=batch_size)
    pbar = tqdm(total=len(docs), desc=" Embeddings for documents")
    model = model.to(device).eval()
    for chunk in it:
        if labels is not None:
            raw_docs, raw_labels = [c[0] for c in chunk], [c[1] for c in chunk]
        else:
            raw_docs = chunk
        _docs = [prefix(x, pref=pref) for x in raw_docs]
        batch_dict = tokenize(_docs, tokenizer=tokenizer)
        batch = {k:v.to(device) for k, v in batch_dict.items()}
        with torch.no_grad():
            outputs = model(**batch)
            _embeddings = average_pool(outputs.last_hidden_state, batch['attention_mask'])
        if norm:
            _embeddings = F.normalize(_embeddings, p=2, dim=len(_embeddings.shape) - 1)
        _embeddings = _embeddings.cpu().numpy()
        if labels is not None:
            chunk = [Document.from_dict({"content": doc, "embedding": list(emb), "labels": [generate_unique_id(label)]}) for doc, label, emb in zip(raw_docs, raw_labels, _embeddings)]
        else:
            chunk = [Document.from_dict({"content": doc, "embedding": list(emb)}) for doc, emb in zip(raw_docs, _embeddings)]
        wrapped_docs.extend(chunk)
        pbar.update(n=len(raw_docs))
    return wrapped_docs

In [18]:
from yasearch.storing.weaviate import WeaviateDocStore
store = WeaviateDocStore(url="http://localhost:2211", collection_name="justatom")

In [19]:
store.count_documents()

0

In [20]:
wrapped_docs = ignite_vectors_by(model=model, docs=documents, labels=labels, pref="passage:", device="cuda")

 Embeddings for documents: 100%|██████████| 616/616 [00:06<00:00, 92.50it/s] 


In [24]:
wrapped_docs[4].content

'Когда-то он был в нашей команде. Его прозвали “Баба-Яга”. Джон - человек целеустремленный, обязательный, волевой. Тебе все это знакомо очень мало... Я видел однажды, как он убил троих в баре... Карандашом. Обычным карандашом. и, вдруг, в один прекрасный день, он захотел уйти. Все из-за женщины, конечно. Поэтому, я заключил с ним сделку. Я дал ему невыполнимое задание - работу, которую никто не смог потянуть. Те, которых он закопал тогда легли в основу нашей организации.\nА теперь мой сын через несколько дней после смерти его жены стырил его тачку и убил его щенка...\nЙозеф, Йозеф, мой сын. Джон придет за тобой и ты ничего не сможешь сделать потому что не сможешь.'

In [25]:
assert wrapped_docs[0].meta["labels"][0] == generate_unique_id(labels[0]), f"Unexptected error due to labeling errors"

In [26]:
wrapped_docs[0].meta

{'labels': ['efc12637-0e91-581d-bdba-64fba3ec6a04']}

In [27]:
store.write_documents(documents=wrapped_docs)

616

In [28]:
queries = [
    "Какие правила голодных игр?",
    "Баба Яга",
    "Любовь"
]

In [31]:
query_vecs = ignite_vectors_by(model=model, docs=queries, pref="query: ")

 Embeddings for documents: 100%|██████████| 3/3 [00:00<00:00,  5.33it/s]


In [32]:
print(query_vecs[0].content)
print(query_vecs[0].embedding)

Какие правила голодных игр?
[0.053049717, 0.04988152, 0.006537093, 0.05279767, 0.049019407, -0.025354119, -0.016592987, -0.05835095, 0.043995444, 0.029732507, -0.0053636367, 0.028418243, 0.17047888, 0.037980232, -0.04613134, -0.053106558, 0.02804633, 0.0067098793, 0.017089207, 0.026184045, 0.016329052, -0.035786778, 0.017885787, -0.006323317, 0.025756037, -0.036064167, 0.009998747, 0.03402225, -0.023247983, 0.024673825, 0.030762702, -0.054473188, -0.0002622584, 0.021900948, 0.024026586, 0.02106541, 0.002189432, -0.016787514, 0.0028538867, 0.025357595, -0.012233675, -0.0133498525, 0.05426145, -0.06845884, 0.008962782, -0.01997723, 0.030984353, -0.0045971186, -0.05502491, -0.047476932, 0.0017278417, 0.02520688, 0.042845547, 0.021489842, -0.05621179, -0.06356001, 0.03876095, 0.028062996, -0.05360235, 0.031846154, 0.0027325242, 0.05696302, -0.015343976, 0.05477503, 0.044966668, -0.018142356, 0.032350186, -0.042801052, -0.040873304, -0.0044993623, -0.014831008, -0.0006840441, 0.039676134, 0

#### Seach `by embedding` only

In [33]:
store.search_by_embedding(query_embedding=query_vecs[0].embedding, top_k=2)

[<Document: {'content': 'Проигрывать всегда горько. Неистово верить в свою правоту, и всё равно проиграть.', 'content_type': 'text', 'score': 0.9016470909118652, 'meta': {'labels': [UUID('f5bced10-6b44-5ba4-bbc0-93ce7cf23f4e')], 'dataframe': None}, 'embedding': '<embedding of shape [no shape]>', 'id': '37c3fd8d7296de6e7f60f63073044af2'}>,
 <Document: {'content': 'В Дистрикте-12 голодная смерть не редкость. За примерами далеко ходить не надо: старики, не способные больше работать, дети из семей, где слишком много ртов, рабочие, искалеченные в шахтах. Бродил вчера человек по улицам, а сегодня, смотришь, лежит где-нибудь, привалившись к забору, и не шевелится. Или на Луговине наткнешься. А другой раз только плач из домов слышишь. Приедут миротворцы, заберут тело. Власти не признают, что это из-за голода. Официально причина всегда – грипп, переохлаждение или воспаление легких.', 'content_type': 'text', 'score': 0.9013684391975403, 'meta': {'labels': [UUID('721626b0-f98a-55d8-b782-9dd6c2185

#### Search `by bm25` only

In [34]:
store.search_by_keywords(query=queries[0], top_k=2)

[<Document: {'content': 'Герой наш поворотился в ту ж минуту к губернаторше и уже готов был отпустить ей ответ, вероятно ничем не хуже тех, какие отпускают в модных повестях Звонские, Линские, Лидины, Гремины и всякие ловкие военные люди, как, невзначай поднявши глаза, остановился вдруг, будто оглушенный ударом.', 'content_type': 'text', 'score': 1.8574644327163696, 'meta': {'labels': [UUID('f9507682-c01d-5f7b-8cb4-437ba795d921')], 'dataframe': None}, 'embedding': None, 'id': 'b3d5e98f550b97faed8dca802838910e'}>,
 <Document: {'content': '– Будут какие-нибудь советы? – интересуется Пит.\n– Как только ударят в гонг, скорее уносите ноги. Мясорубка перед Рогом изобилия вам не по зубам. Улепетывайте что есть духу, чем дальше от других, тем лучше, и ищите источник воды. Ясно?\n– А потом? – спрашиваю я.\n– А потом постарайтесь выжить, – отвечает Хеймитч.', 'content_type': 'text', 'score': 1.8173242807388306, 'meta': {'labels': [UUID('721626b0-f98a-55d8-b782-9dd6c2185bd6')], 'dataframe': None}

#### Search by both `bm25` and `embedding` using `alpha` score to merge the rankings from both

In [35]:
alpha = 0.6
query = query_vecs[0].content
query_emb = query_vecs[0].embedding

response = store.search(query=query, query_embedding=query_emb, alpha=alpha, top_k=2)

response_docs = "\n---".join([f"\nDOC[{str(pos)}]\n" + x.content for pos, x in enumerate(response)])
logger.info(f" >> | {query}")
logger.info(response_docs)

2024-06-15 21:54:44.221 | INFO     | __main__:<module>:8 -  >> | Какие правила голодных игр?
2024-06-15 21:54:44.222 | INFO     | __main__:<module>:9 - 
DOC[0]
– Будут какие-нибудь советы? – интересуется Пит.
– Как только ударят в гонг, скорее уносите ноги. Мясорубка перед Рогом изобилия вам не по зубам. Улепетывайте что есть духу, чем дальше от других, тем лучше, и ищите источник воды. Ясно?
– А потом? – спрашиваю я.
– А потом постарайтесь выжить, – отвечает Хеймитч.
---
DOC[1]
Проигрывать всегда горько. Неистово верить в свою правоту, и всё равно проиграть.
